# Python | Урок 3: построение моделей

## Загрузка необходимых библиотек

Для работы нам понадобятся две библиотеки:
1. `pandas`: при помощи нее мы считаем из файла таблицу с данными
3. `sklearn`: библиотека инструментов для анализа данных и машинного обучения

In [1]:
# Подключаем нужные библиотеки
import pandas as pd
import sklearn

In [2]:
# Прочитаем данные уже известным нам методом
table = pd.read_csv('dota2_skill_train.csv', index_col='id')

# и поделим данные на признаки и целевую переменную
x = table.drop(['skilled'], axis = 1)
y = table['skilled']

По традиции проверим, что все считалось корректно

In [3]:
# Выводим первые 5 строк таблицы признаков
x.head()

,player_team,winner_team,duration,pre_game_duration,first_blood_time,first_blood_claimed,hero_id,hero_pick_order,leaver_status,party_players,...,avg_deaths_x16,avg_assists_x16,avg_gpm_x16,avg_xpm_x16,best_kills_x16,best_assists_x16,best_gpm_x16,best_xpm_x16,win_streak,best_win_streak
id,,,,,,,,,,,,,,,,,,,,,
7,dire,dire,2140,90,129,0,90,9,0,3,...,8,15,352,430,10,30,551,745,2,5
8,radiant,radiant,2138,90,174,0,5,5,0,1,...,9,19,294,425,13,37,445,717,2,12
12,radiant,radiant,3547,90,360,0,81,7,0,1,...,7,9,493,543,23,18,691,762,3,3
13,dire,radiant,1878,90,28,0,74,9,0,2,...,7,12,515,583,25,34,869,935,0,6
14,dire,radiant,2232,90,129,0,14,6,0,2,...,10,16,337,452,34,43,672,797,0,9


In [4]:
# и первые 5 строк столбца целевой переменной
y.head()

id
7     1
8     0
12    0
13    1
14    1
Name: skilled, dtype: int64

Для задач классификации, как наша, очень важно знать - сбалансированы классы или нет. Другими словами: какое соотношение объектов разных классов в обучающей выборке?

In [9]:
# Для того, чтобы это проверить, используем метод value_counts() объекта y 
y.value_counts()

0    60085
1    39786
Name: skilled, dtype: int64

Как видите, объектов класса "0" (неопытные игроки) в полтора раза больше, чем объектов класса "1" (опытные игроки). Это не самый большой дисбаланс классов, поэтому не волнуйтесь :)

In [10]:
# Также важно проверить, нет ли пропусков в данных
count_missings = x.isnull().sum()
count_missings[count_missings > 0]

Series([], dtype: int64)

Какое везение! Все данные на месте, пропусков нет. В реальных задача такое бывает редко, но здесь организаторы соревнования о вас позаботились и подготовили очень хороший набор данных.

In [11]:
# Подготовим данные для алгоритма
# Заменим строковые значения в колонках 'player_team' и 'winner_team' на числовые
x['player_team'] = x['player_team'].apply(lambda x: 1 if x == 'dire' else 0)
x['winner_team'] = x['winner_team'].apply(lambda x: 1 if x == 'dire' else 0)

## Построение модели

In [12]:
# Для начала разбиваем выборку на обучающую и валидационную (проверочную)

# Загружаем нужный модуль: train_test_split 
from sklearn.model_selection import train_test_split

# Создаем 4 новых переменных разделив выборки x и y по строкам в соотношении 2 к 1 (размер тестовой выборки = 0.33)
# 2 для обучения модели (с префиксом _train) 
# и 2 для проверки (с префиксом _valid) 

x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=.33, random_state=1)

Как вы уже знаете, у каждого алгоритма машинного обучения есть масса параметров. Мы будем использовать алгоритм RandomForestClassifier - этот "лес" является ансамблем большого количества деревьев решений, о которых мы говорили в видеоуроках.   

Чтобы не подбирать его параметры вручную, воспользуемся готовой библиотекой GridSearcCV, которой нужно передать набор значений какого-либо параметра (или нескольких) и дождаться результата.

In [13]:
# Загружаем классификатор RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

# Загружаем GridSearchCV для подбора параметров
from sklearn.model_selection import GridSearchCV

# Задаем 'сетку' возможных значений - количество деревьев решений в "лесу"
param_grid = {'n_estimators': [10, 50, 100]}

# Создаем объект, в котором для нашего классификатора будет идти поиск лучших параметров
clf = GridSearchCV(RandomForestClassifier(random_state=322), param_grid, verbose=3, n_jobs=-1)

# Обучаем этот объект на подготовленных данных
clf.fit(x_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  15 | elapsed:   16.0s remaining:   24.1s
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:   31.4s remaining:    7.8s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   39.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=322,
                              

In [14]:
# Итак, подбор параметров окончен, посмотрим на "лучший" по мнению GridSearchCV классификатор
best_clf = clf.best_estimator_
best_clf

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=322,
                       verbose=0, warm_start=False)

Обратите внимание на параметр 'n_estimators=100' - получилось, что 100 деревьев самый лучшей вариант среди `[10, 50, 100]`.

### Оценим качество модели

In [15]:
# Теперь сделаем предсказание переменной 'skilled' на валидационной выборке и оценим качество модели

# Загружаем модуль accuracy_score - который посчитает нам долю правильных ответов, 
# сравнив истинные значения y и предсказанные
from sklearn.metrics import accuracy_score

# Считаем долю правильных ответов
validation_acc = accuracy_score(y_validation, best_clf.predict(x_validation))

# Выводим на экран
print('Точность на валидации:', validation_acc)

Точность на валидации: 0.7124218702591176


Это значение говорит о том, что данный классификатор "угадал" 71% правильных ответов - согласитесь, не очень хороший предсказатель. Что же делать?

Сейчас, когда вы познакомились с основными библиотеками python для машинного обучения, самое время переходить к прорешиванию [базового решения соревнования](https://github.com/sberbank-ai/ai-academy-2019/blob/master/Dota2SkillPrediction_Tutorial.ipynb)! 

C его помощью вы сможете сделать свой первый сабмит, а также, немного усовершенствов, занять достойное место в рейтинге и попасть в финал!